# 计算Firm Object
参数是用的两个扩散过程中的参数。


$k\in[0.9,0.98]   $
$p\in[0.4,0.6]$

## 运行过程

In [4]:
import numpy as np

kmin=0.9
kmax=0.98
pmin=0.4
pmax=0.6
kk=np.linspace(kmin,kmax,10)#行向量
pp=np.linspace(pmin,pmax,10)
kk=kk.reshape((1,10))
pp=pp.reshape((1,10))
#print(k)
ones=np.ones((10,1))
#print(ones)
kk=np.dot(ones,kk)
pp=np.dot(ones,pp)
# print(k)
kk=np.transpose(kk)
kk=kk.reshape((100,1))
pp=pp.reshape((100,1))
# print(k)
Parameter=np.hstack((kk,pp))
# print(Parameter)

# for i in range(0,100,):
#     k=Parameter[i,0]
#     p=Parameter[i,1]
FirmWelfare=np.zeros((100,1))

In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys
from scipy.sparse import spdiags
from datetime import datetime
from openpyxl import load_workbook
from scipy.sparse import identity
import copy


for i in range(0,100,1):
    k=Parameter[i,0];
    p=Parameter[i,1];
    gamma=0.9;
    epsilon=-0.35;

    rho=0.9;
    C=50;
    # k=0.6;
    # p=0.5;

    a=0.8;
    eta=0.6;
    xmean=a/eta;
    sigma=0.8;
    sd=sigma**2/2/eta
    xmin=xmean-2*sd
    xmax=xmean+2*sd

    I=1000;
    hmin=0.01;
    hmax=100;
    h=np.linspace(hmin,hmax,I);   #列向量  
                                  #linspace(start,stop,num)生成array对象,num是点的数量，包括两端
    h=h.reshape(I,1);
    #print(h);
    dh=(hmax-hmin)/(I-1);

    J=7;
    x=np.linspace(xmin,xmax,J);    # 行向量
    x=x.reshape(1,J);    #此时x的输出结果为[[2  6  5  4]]，可以用x[0][1]来调取
    dx=(xmax-xmin)/(J-1);
    dx2=dx**2;

    hh=np.dot(h,np.ones((1,J)));
    xx=np.dot(np.ones((I,1)),x);

    mu=a-x*eta;      
    s2=0.5*sigma**2;

    Vhf = np.zeros((I,J));
    Vhb = np.zeros((I,J));
    Vxf = np.zeros((I,J));
    Vxb = np.zeros((I,J));
    Vxx = np.zeros((I,J));
    n = np.zeros((I,J));

    chi =  - np.minimum(mu,0)/dx + s2/dx2;              
    yy =  np.minimum(mu,0)/dx - np.maximum(mu,0)/dx - s2*2/dx2;
    zeta = np.maximum(mu,0)/dx + s2/dx2;

    maxit=100;
    crit=10**(-6);
    Delta=1000;

    updiag=np.zeros((I,1));
    for j in range(0,J,1):
        gg=np.tile(zeta[0][j],(I,1));
        updiag=np.vstack([updiag,gg]);

    centdiag=np.tile(chi[0][0]+yy[0][0],(I,1))
    for j in range(1,J-1,1):
        gg=np.tile(yy[0][j],(I,1));
        centdiag=np.vstack([centdiag,gg]);
    gg=np.tile(yy[0][J-1]+zeta[0][J-1],(I,1))
    centdiag=np.vstack([centdiag,gg])

    lowdiag=np.tile(chi[0][1],(I,1));
    for j in range(2,J,1):
        gg=np.tile(chi[0][j],(I,1));
        lowdiag=np.vstack([lowdiag,gg]);

    centdiag=np.transpose(centdiag);
    Bswitch_centdiag=spdiags(centdiag,0,I*J,I*J).toarray();
    lowdiag=np.transpose(lowdiag);
    Bswitch_lowdiag=spdiags(lowdiag,-I,I*J,I*J).toarray();
    updiag=np.transpose(updiag);
    Bswitch_updiag=spdiags(updiag,I,I*J,I*J).toarray();
    Bswitch=Bswitch_centdiag+Bswitch_lowdiag+Bswitch_updiag;

    v0=(xx*hh**(1-gamma)/(1-gamma))/rho;
    v=v0;

    maxit=30;
    distm=np.zeros(maxit);
    for m in range(0,maxit,1):   
        V=copy.deepcopy(v);
        Vhf=np.zeros((I,J));
        Vhf[0:I-1,:]=(V[1:I,:]-V[0:I-1,:])/dh;
        Vhf[I-1,:]=(V[I-1,:]-V[I-2,:])/dh;

        Vhb=np.zeros((I,J));
        Vhb[1:I,:]=(V[1:I,:]-V[0:I-1,:])/dh;
        Vhb[0,:]=(V[1,:]-V[0,:])/dh;

        I_concave=(Vhb>Vhf)+0;

        nf=((1-k)/C*Vhf)**epsilon;
        hf=(1-k)*nf-p*hh;

        nb=((1-k)/C*Vhb)**epsilon;
        hb=(1-k)*nb-p*hh;

        n0=p/(1-k)*hh;
        Vh0=C/(1-k)*n0**(1/epsilon);

        If=(hf>0)+0;
        Ib=(hb<0)+0;

        I0=(1-If-Ib);

        Vh_Upwind=Vhf*If+Vhb*Ib+Vh0*I0;

        n=((1-k)/C*Vh_Upwind)**epsilon;
        u=xx*hh**(1-gamma)/(1-gamma)-C/(1+1/epsilon)*n**(1+1/epsilon);

    #     where_are_nan = np.isnan(hb)
    #     hb[where_are_nan] = 0
    #     where_are_nan = np.isnan(hf)
    #     hf[where_are_nan] = 0

        X=-np.minimum(hb,0)/dh;
        Y=-np.maximum(hf,0)/dh+np.minimum(hb,0)/dh;
        Z=np.maximum(hf,0)/dh;

        updiag=0;
        for j in range(0,J,1):
            pp=Z[0:I-1,j];
            pp=pp.reshape(I-1,1)
            gg=np.vstack([pp,0]);
            updiag=np.vstack([updiag,gg]);

        Y=np.transpose(Y);
        centdiag=Y.reshape(I*J,1);   #按行来reshape    python中的reshape是按行的，但此处我需要按列来，所以转置

        lowdiag=X[1:I,0];
        lowdiag=lowdiag.reshape(I-1,1);
        for j in range(1,J,1):
            pp=X[1:I,j];
            pp=pp.reshape(I-1,1);
            gg=np.vstack([0,pp]);
            lowdiag=np.vstack([lowdiag,gg]);

        centdiag=np.transpose(centdiag);
        AA_centdiag=spdiags(centdiag,0,I*J,I*J).toarray();

        lowdiag=np.vstack([lowdiag,0]);
        lowdiag=np.transpose(lowdiag);
        AA_lowdiag=spdiags(lowdiag,-1,I*J,I*J).toarray();

        updiag=np.vstack([updiag,0]);
        updiag=np.transpose(updiag);
        AA_updiag=spdiags(updiag,1,I*J,I*J).toarray();

        AA=AA_centdiag+AA_lowdiag+AA_updiag;

        A=AA+Bswitch;

        if np.max(abs(np.sum(A,axis=1)))>10**(-9):   #这个最大值不太一样    
            print("Improper Transition Matrix")
            break;

        B=(1/Delta+rho)*identity(I*J).toarray()-A;
        invB=np.linalg.inv(B);

        u=np.transpose(u);
        u_stacked=copy.deepcopy(u.reshape(I*J,1));
        V=np.transpose(V);
        V_stacked=copy.deepcopy(V.reshape(I*J,1));


        b=u_stacked+V_stacked/Delta;

        invBcopy=copy.deepcopy(invB);
        bcopy=np.tile(b,(1,I*J))

#         for j in range(0,I*J,1):
#             for i in range(0,I*J,1):
#                 if (np.isnan(bcopy[i,j])==1)|(bcopy[i,j]==np.inf):
#                     if invBcopy[j,i]==0:
#                         bcopy[i,j]=0;
#         V_stacked2=np.zeros((I*J,1))
#         for j in range(0,I*J,1):
#             V_stacked2[j,0]=np.dot(invBcopy[j,:],bcopy[:,j])
        V_stacked2=np.dot(invBcopy,b)

        V=copy.deepcopy(V_stacked2.reshape(J,I));
        V=np.transpose(V);  #matlab是按列来进行reshape，python按行来reshape

        Vchange=V-v;

        v=copy.deepcopy(V);

        Vchangecopy=copy.deepcopy(Vchange)
    #     where_are_nan = np.isnan(Vchangecopy)
    #     where_are_inf = np.isinf(Vchangecopy)
    #     Vchangecopy[where_are_nan] = 0
    #     Vchangecopy[where_are_inf] = 0

        distm[m]=np.max(np.max(abs(Vchangecopy),axis=0));
        print(distm[m]);
        print(m);
        if distm[m]<crit:
            print("Value Function Converged, Iteration =");
            print(m);
            print(distm[m]);
            break;
            
    AT=np.transpose(A)
    b5=np.zeros((I*J,1))
    i_fix=5
    b5[i_fix-1,0]=0.1
    row=np.hstack([np.zeros(i_fix-1),1,np.zeros(I*J-i_fix)])
    AT[i_fix-1,:]=row
    invAT=np.linalg.inv(AT);
    gg=np.dot(invAT,b5);

    g_sum=np.dot(np.transpose(gg),np.ones((I*J,1)))*dh*dx

    gg=gg/g_sum
    # g=np.transpose(gg).reshape(I,J)
    ggcopy=copy.deepcopy(gg)
    # ggcopy=np.transpose(ggcopy)
    g=ggcopy.reshape(J,I)
    g=np.transpose(g)
    
    
    T=k*n+p*hh
    
    Tg=T*g
    TFW1=np.dot(np.ones((1,I)),Tg)
    TotalFirmWelfare=np.dot(TFW1,np.ones((J,1)))
    print('k=',k,'p=',p,'FirmWelfare=',TotalFirmWelfare)
    FirmWelfare[i,0]=TotalFirmWelfare

11.365096106576726
0
0.6971526132678676
1
0.04238883796363524
2
0.0003367256296051835
3
1.0631226921020698e-07
4
Value Function Converged, Iteration =
4
1.0631226921020698e-07
k= 0.9 p= 0.4 FirmWelfare= [[358.76123239]]
11.42466357038839
0
0.7010156894003714
1
0.04286496056334954
2
0.00035078445926473023
3
1.1466694438411196e-07
4
Value Function Converged, Iteration =
4
1.1466694438411196e-07
k= 0.9 p= 0.4222222222222222 FirmWelfare= [[353.05472496]]
11.483884167602522
0
0.7048866254185171
1
0.04334158037810454
2
0.00036531591462996005
3
1.236981663765846e-07
4
Value Function Converged, Iteration =
4
1.236981663765846e-07
k= 0.9 p= 0.4444444444444445 FirmWelfare= [[347.96978163]]
11.542757877422641
0
0.7087642029369476
1
0.043818523457833436
2
0.0003803416529581938
3
1.3347990979184488e-07
4
Value Function Converged, Iteration =
4
1.3347990979184488e-07
k= 0.9 p= 0.4666666666666667 FirmWelfare= [[343.16060184]]
11.601284687970224
0
0.7126520129617546
1
0.04429521161990024
2
0.000395855

11.635471810563423
0
0.5806133030529175
1
0.021084481889356965
2
6.168316511612204e-05
3
1.392801074473482e-08
4
Value Function Converged, Iteration =
4
1.392801074473482e-08
k= 0.9266666666666666 p= 0.4888888888888889 FirmWelfare= [[344.03992004]]
11.694144817196086
0
0.5842037330042871
1
0.021319294242086784
2
6.362031512985311e-05
3
1.4343921606041476e-08
4
Value Function Converged, Iteration =
4
1.4343921606041476e-08
k= 0.9266666666666666 p= 0.5111111111111111 FirmWelfare= [[339.92872762]]
11.752481227419992
0
0.5877680568583337
1
0.021560188019510207
2
6.561626279300015e-05
3
1.4820347615795981e-08
4
Value Function Converged, Iteration =
4
1.4820347615795981e-08
k= 0.9266666666666666 p= 0.5333333333333333 FirmWelfare= [[336.11629267]]
11.810481092078945
0
0.5913847627801232
1
0.021791291529037693
2
6.761567825641634e-05
3
1.51737555853515e-08
4
Value Function Converged, Iteration =
4
1.51737555853515e-08
k= 0.9266666666666666 p= 0.5555555555555556 FirmWelfare= [[332.84180003]]
11

6.4752440458448746e-06
3
9.92672610777845e-09
4
Value Function Converged, Iteration =
4
9.92672610777845e-09
k= 0.9533333333333334 p= 0.5555555555555556 FirmWelfare= [[339.95990521]]
11.91017101705389
0
0.4229269672180429
1
0.007445912670068111
2
6.639510198525045e-06
3
2.420751599174764e-09
4
Value Function Converged, Iteration =
4
2.420751599174764e-09
k= 0.9533333333333334 p= 0.5777777777777777 FirmWelfare= [[337.37424083]]
11.968150097239327
0
0.42604786313421883
1
0.007515919049616571
2
6.807277099341036e-06
3
1.231644652932573e-08
4
Value Function Converged, Iteration =
4
1.231644652932573e-08
k= 0.9533333333333334 p= 0.6 FirmWelfare= [[333.4528131]]
11.448877937841573
0
0.333715021508862
1
0.003997687278499029
2
2.515995653595837e-06
3
1.0028600172518054e-09
4
Value Function Converged, Iteration =
4
1.0028600172518054e-09
k= 0.9622222222222222 p= 0.4 FirmWelfare= [[377.1850769]]
11.50965259726378
0
0.33636494359918245
1
0.004053622046939154
2
2.5722320629029127e-06
3
2.304307855

## 下面是不同对k，p所对应的FirmWelfare

运行了70min，此结果保留，不再改动，太费时间了。2023.3.6

In [6]:
print(FirmWelfare)

[[358.76123239]
 [353.05472496]
 [347.96978163]
 [343.16060184]
 [338.85823622]
 [334.64603128]
 [330.94992091]
 [327.5045427 ]
 [324.28409073]
 [321.3150778 ]
 [360.51258346]
 [354.77785699]
 [349.53114906]
 [344.89640133]
 [340.42729839]
 [336.32268557]
 [332.58908973]
 [329.14009157]
 [325.76830108]
 [322.58417446]
 [362.31355943]
 [356.73695136]
 [351.35247173]
 [346.70467027]
 [342.10082956]
 [338.01490859]
 [334.20824614]
 [330.63704469]
 [327.39179794]
 [324.5150697 ]
 [364.59398261]
 [358.67706087]
 [353.53757949]
 [348.50057063]
 [344.03992004]
 [339.92872762]
 [336.11629267]
 [332.84180003]
 [329.44017698]
 [326.0874601 ]
 [366.9027524 ]
 [360.98305451]
 [355.64648788]
 [350.89508783]
 [346.35991118]
 [342.14778789]
 [338.67259414]
 [334.99837048]
 [331.2686445 ]
 [328.82577561]
 [369.57855056]
 [363.87970441]
 [358.49528453]
 [353.49868477]
 [348.8186048 ]
 [344.89308958]
 [341.34882875]
 [337.15370415]
 [334.43183194]
 [330.71073766]
 [372.99197394]
 [367.12810805]
 [361.95

上面cell的结果，保存在此

[[358.76123239]
 [353.05472496]
 [347.96978163]
 [343.16060184]
 [338.85823622]
 [334.64603128]
 [330.94992091]
 [327.5045427 ]
 [324.28409073]
 [321.3150778 ]
 [360.51258346]
 [354.77785699]
 [349.53114906]
 [344.89640133]
 [340.42729839]
 [336.32268557]
 [332.58908973]
 [329.14009157]
 [325.76830108]
 [322.58417446]
 [362.31355943]
 [356.73695136]
 [351.35247173]
 [346.70467027]
 [342.10082956]
 [338.01490859]
 [334.20824614]
 [330.63704469]
 [327.39179794]
 [324.5150697 ]
 [364.59398261]
 [358.67706087]
 [353.53757949]
 [348.50057063]
 [344.03992004]
 [339.92872762]
 [336.11629267]
 [332.84180003]
 [329.44017698]
 [326.0874601 ]
 [366.9027524 ]
 [360.98305451]
 [355.64648788]
 [350.89508783]
 [346.35991118]
 [342.14778789]
 [338.67259414]
 [334.99837048]
 [331.2686445 ]
 [328.82577561]
 [369.57855056]
 [363.87970441]
 [358.49528453]
 [353.49868477]
 [348.8186048 ]
 [344.89308958]
 [341.34882875]
 [337.15370415]
 [334.43183194]
 [330.71073766]
 [372.99197394]
 [367.12810805]
 [361.95016656]
 [356.92539434]
 [351.90145918]
 [347.96108827]
 [344.23320339]
 [339.95990521]
 [337.37424083]
 [333.4528131 ]
 [377.1850769 ]
 [370.92143821]
 [365.12141704]
 [360.71023541]
 [356.06856316]
 [351.23116697]
 [347.95781789]
 [343.6338437 ]
 [340.72480251]
 [337.48788998]
 [382.09313999]
 [375.61373557]
 [370.8239647 ]
 [365.26028307]
 [360.73601249]
 [356.18464746]
 [352.46799379]
 [348.70752095]
 [345.07793652]
 [341.88770675]
 [389.16700687]
 [383.0184046 ]
 [376.9969502 ]
 [371.74411346]
 [367.38978605]
 [362.61282776]
 [358.75180844]
 [354.31122618]
 [350.58172398]
 [347.43116094]]


## 调试，去掉其中的for循环。

已去掉。

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys
from scipy.sparse import spdiags
from datetime import datetime
from openpyxl import load_workbook
from scipy.sparse import identity
import copy


# for i in range(0,100,1):
# k=Parameter[i,0];
# p=Parameter[i,1];
k=0.9
p=0.4
gamma=0.9;
epsilon=-0.35;

rho=0.9;
C=50;
# k=0.6;
# p=0.5;

a=0.8;
eta=0.6;
xmean=a/eta;
sigma=0.8;
sd=sigma**2/2/eta
xmin=xmean-2*sd
xmax=xmean+2*sd

I=1000;
hmin=0.01;
hmax=100;
h=np.linspace(hmin,hmax,I);   #列向量  
                              #linspace(start,stop,num)生成array对象,num是点的数量，包括两端
h=h.reshape(I,1);
#print(h);
dh=(hmax-hmin)/(I-1);

J=7;
x=np.linspace(xmin,xmax,J);    # 行向量
x=x.reshape(1,J);    #此时x的输出结果为[[2  6  5  4]]，可以用x[0][1]来调取
dx=(xmax-xmin)/(J-1);
dx2=dx**2;

hh=np.dot(h,np.ones((1,J)));
xx=np.dot(np.ones((I,1)),x);

mu=a-x*eta;      
s2=0.5*sigma**2;

Vhf = np.zeros((I,J));
Vhb = np.zeros((I,J));
Vxf = np.zeros((I,J));
Vxb = np.zeros((I,J));
Vxx = np.zeros((I,J));
n = np.zeros((I,J));

chi =  - np.minimum(mu,0)/dx + s2/dx2;              
yy =  np.minimum(mu,0)/dx - np.maximum(mu,0)/dx - s2*2/dx2;
zeta = np.maximum(mu,0)/dx + s2/dx2;

maxit=100;
crit=10**(-6);
Delta=1000;

updiag=np.zeros((I,1));
for j in range(0,J,1):
    gg=np.tile(zeta[0][j],(I,1));
    updiag=np.vstack([updiag,gg]);

centdiag=np.tile(chi[0][0]+yy[0][0],(I,1))
for j in range(1,J-1,1):
    gg=np.tile(yy[0][j],(I,1));
    centdiag=np.vstack([centdiag,gg]);
gg=np.tile(yy[0][J-1]+zeta[0][J-1],(I,1))
centdiag=np.vstack([centdiag,gg])

lowdiag=np.tile(chi[0][1],(I,1));
for j in range(2,J,1):
    gg=np.tile(chi[0][j],(I,1));
    lowdiag=np.vstack([lowdiag,gg]);

centdiag=np.transpose(centdiag);
Bswitch_centdiag=spdiags(centdiag,0,I*J,I*J).toarray();
lowdiag=np.transpose(lowdiag);
Bswitch_lowdiag=spdiags(lowdiag,-I,I*J,I*J).toarray();
updiag=np.transpose(updiag);
Bswitch_updiag=spdiags(updiag,I,I*J,I*J).toarray();
Bswitch=Bswitch_centdiag+Bswitch_lowdiag+Bswitch_updiag;

v0=(xx*hh**(1-gamma)/(1-gamma))/rho;
v=v0;

maxit=30;
distm=np.zeros(maxit);
for m in range(0,maxit,1):   
    V=copy.deepcopy(v);
    Vhf=np.zeros((I,J));
    Vhf[0:I-1,:]=(V[1:I,:]-V[0:I-1,:])/dh;
    Vhf[I-1,:]=(V[I-1,:]-V[I-2,:])/dh;

    Vhb=np.zeros((I,J));
    Vhb[1:I,:]=(V[1:I,:]-V[0:I-1,:])/dh;
    Vhb[0,:]=(V[1,:]-V[0,:])/dh;

    I_concave=(Vhb>Vhf)+0;

    nf=((1-k)/C*Vhf)**epsilon;
    hf=(1-k)*nf-p*hh;

    nb=((1-k)/C*Vhb)**epsilon;
    hb=(1-k)*nb-p*hh;

    n0=p/(1-k)*hh;
    Vh0=C/(1-k)*n0**(1/epsilon);

    If=(hf>0)+0;
    Ib=(hb<0)+0;

    I0=(1-If-Ib);

    Vh_Upwind=Vhf*If+Vhb*Ib+Vh0*I0;

    n=((1-k)/C*Vh_Upwind)**epsilon;
    u=xx*hh**(1-gamma)/(1-gamma)-C/(1+1/epsilon)*n**(1+1/epsilon);

#     where_are_nan = np.isnan(hb)
#     hb[where_are_nan] = 0
#     where_are_nan = np.isnan(hf)
#     hf[where_are_nan] = 0

    X=-np.minimum(hb,0)/dh;
    Y=-np.maximum(hf,0)/dh+np.minimum(hb,0)/dh;
    Z=np.maximum(hf,0)/dh;

    updiag=0;
    for j in range(0,J,1):
        pp=Z[0:I-1,j];
        pp=pp.reshape(I-1,1)
        gg=np.vstack([pp,0]);
        updiag=np.vstack([updiag,gg]);

    Y=np.transpose(Y);
    centdiag=Y.reshape(I*J,1);   #按行来reshape    python中的reshape是按行的，但此处我需要按列来，所以转置

    lowdiag=X[1:I,0];
    lowdiag=lowdiag.reshape(I-1,1);
    for j in range(1,J,1):
        pp=X[1:I,j];
        pp=pp.reshape(I-1,1);
        gg=np.vstack([0,pp]);
        lowdiag=np.vstack([lowdiag,gg]);

    centdiag=np.transpose(centdiag);
    AA_centdiag=spdiags(centdiag,0,I*J,I*J).toarray();

    lowdiag=np.vstack([lowdiag,0]);
    lowdiag=np.transpose(lowdiag);
    AA_lowdiag=spdiags(lowdiag,-1,I*J,I*J).toarray();

    updiag=np.vstack([updiag,0]);
    updiag=np.transpose(updiag);
    AA_updiag=spdiags(updiag,1,I*J,I*J).toarray();

    AA=AA_centdiag+AA_lowdiag+AA_updiag;

    A=AA+Bswitch;

    if np.max(abs(np.sum(A,axis=1)))>10**(-9):   #这个最大值不太一样    
        print("Improper Transition Matrix")
        break;

    B=(1/Delta+rho)*identity(I*J).toarray()-A;
    invB=np.linalg.inv(B);

    u=np.transpose(u);
    u_stacked=copy.deepcopy(u.reshape(I*J,1));
    V=np.transpose(V);
    V_stacked=copy.deepcopy(V.reshape(I*J,1));


    b=u_stacked+V_stacked/Delta;

    invBcopy=copy.deepcopy(invB);


    V_stacked2=np.dot(invBcopy,b)
    
#     V_stacked2=np.zeros((I*J,1))
#     for j in range(0,I*J,1):
#         V_stacked2[j,0]=np.dot(invBcopy[j,:],bcopy[:,j])

    V=copy.deepcopy(V_stacked2.reshape(J,I));
    V=np.transpose(V);  #matlab是按列来进行reshape，python按行来reshape

    Vchange=V-v;

    v=copy.deepcopy(V);

    Vchangecopy=copy.deepcopy(Vchange)
#     where_are_nan = np.isnan(Vchangecopy)
#     where_are_inf = np.isinf(Vchangecopy)
#     Vchangecopy[where_are_nan] = 0
#     Vchangecopy[where_are_inf] = 0

    distm[m]=np.max(np.max(abs(Vchangecopy),axis=0));
    print(distm[m]);
    print(m);
    if distm[m]<crit:
        print("Value Function Converged, Iteration =");
        print(m);
        print(distm[m]);
        break;

#     AT=np.transpose(A)
#     b5=np.zeros((I*J,1))
#     i_fix=5
#     b5[i_fix-1,0]=0.1
#     row=np.hstack([np.zeros(i_fix-1),1,np.zeros(I*J-i_fix)])
#     AT[i_fix-1,:]=row
#     invAT=np.linalg.inv(AT);
#     gg=np.dot(invAT,b5);

#     g_sum=np.dot(np.transpose(gg),np.ones((I*J,1)))*dh*dx

#     gg=gg/g_sum
#     # g=np.transpose(gg).reshape(I,J)
#     ggcopy=copy.deepcopy(gg)
#     # ggcopy=np.transpose(ggcopy)
#     g=ggcopy.reshape(J,I)
#     g=np.transpose(g)
    
    
#     T=k*n+p*hh
    
#     Tg=T*g
#     TFW1=np.dot(np.ones((1,I)),Tg)
#     TotalFirmWelfare=np.dot(TFW1,np.ones((J,1)))
#     print('k=',k,'p=',p,'FirmWelfare=',TotalFirmWelfare)
#     FirmWelfare[i,0]=TotalFirmWelfare

11.365096106576726
0


KeyboardInterrupt: 

# 优化后的代码，可以从h=0开始做，并且较快，用的参数是h不扩散时的参数


k和p的范围可能需要再调一下。tranlate.m中的k，p值总是显示奇异

## 运行过程


In [9]:
import numpy as np

kmin=0.9
kmax=0.98
pmin=0.4
pmax=0.6
kk=np.linspace(kmin,kmax,10)#行向量
pp=np.linspace(pmin,pmax,10)
kk=kk.reshape((1,10))
pp=pp.reshape((1,10))
#print(k)
ones=np.ones((10,1))
#print(ones)
kk=np.dot(ones,kk)
pp=np.dot(ones,pp)
# print(k)
kk=np.transpose(kk)
kk=kk.reshape((100,1))
pp=pp.reshape((100,1))
# print(k)
Parameter=np.hstack((kk,pp))
# print(Parameter)

# for i in range(0,100,):
#     k=Parameter[i,0]
#     p=Parameter[i,1]
FirmWelfare=np.zeros((100,1))

In [10]:
%%time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys
from scipy.sparse import spdiags
from datetime import datetime
from openpyxl import load_workbook
from scipy.sparse import identity
import copy


for i in range(0,100,1):
    k=Parameter[i,0];
    p=Parameter[i,1];

    gamma=3;
    epsilon=-0.2;

    rho=0.9;
    C=10;

    a=1;
    eta=0.6;
    xmean=a/eta;
    sigma=0.8;
    sd=sigma**2/2/eta
    xmin=xmean-2*sd
    xmax=xmean+2*sd

    I=1000;
    hmin=0;
    hmax=100;
    h=np.linspace(hmin,hmax,I);  #列向量  
                                  #linspace(start,stop,num)生成array对象,num是点的数量，包括两端
    h=h.reshape(I,1);
    #print(h);
    dh=(hmax-hmin)/(I-1);

    J=7;
    x=np.linspace(xmin,xmax,J);    # 行向量
    x=x.reshape(1,J);    #此时x的输出结果为[[2  6  5  4]]，可以用x[0][1]来调取
    dx=(xmax-xmin)/(J-1);
    dx2=dx**2;

    hh=np.dot(h,np.ones((1,J)));
    xx=np.dot(np.ones((I,1)),x);

    mu=a-x*eta;      
    s2=0.5*sigma**2;

    Vhf = np.zeros((I,J));
    Vhb = np.zeros((I,J));
    Vxf = np.zeros((I,J));
    Vxb = np.zeros((I,J));
    Vxx = np.zeros((I,J));
    n = np.zeros((I,J));

    chi =  - np.minimum(mu,0)/dx + s2/dx2;              
    yy =  np.minimum(mu,0)/dx - np.maximum(mu,0)/dx - s2*2/dx2;
    zeta = np.maximum(mu,0)/dx + s2/dx2;

    maxit=100;
    crit=10**(-6);
    Delta=1000;

    updiag=np.zeros((I,1));
    for j in range(0,J,1):
        gg=np.tile(zeta[0][j],(I,1));
        updiag=np.vstack([updiag,gg]);

    centdiag=np.tile(chi[0][0]+yy[0][0],(I,1))
    for j in range(1,J-1,1):
        gg=np.tile(yy[0][j],(I,1));
        centdiag=np.vstack([centdiag,gg]);
    gg=np.tile(yy[0][J-1]+zeta[0][J-1],(I,1))
    centdiag=np.vstack([centdiag,gg])

    lowdiag=np.tile(chi[0][1],(I,1));
    for j in range(2,J,1):
        gg=np.tile(chi[0][j],(I,1));
        lowdiag=np.vstack([lowdiag,gg]);

    centdiag=np.transpose(centdiag);
    Bswitch_centdiag=spdiags(centdiag,0,I*J,I*J).toarray();
    lowdiag=np.transpose(lowdiag);
    Bswitch_lowdiag=spdiags(lowdiag,-I,I*J,I*J).toarray();
    updiag=np.transpose(updiag);
    Bswitch_updiag=spdiags(updiag,I,I*J,I*J).toarray();
    Bswitch=Bswitch_centdiag+Bswitch_lowdiag+Bswitch_updiag;

    v0=(xx*hh**(1-gamma)/(1-gamma))/rho;
    v=v0;

    maxit=30;
    distm=np.zeros(maxit);
    for m in range(0,maxit,1):   
        V=copy.deepcopy(v);
        Vhf=np.zeros((I,J));
        Vhf[0:I-1,:]=(V[1:I,:]-V[0:I-1,:])/dh;
        Vhf[I-1,:]=(V[I-1,:]-V[I-2,:])/dh;

        Vhb=np.zeros((I,J));
        Vhb[1:I,:]=(V[1:I,:]-V[0:I-1,:])/dh;
        Vhb[0,:]=(V[1,:]-V[0,:])/dh;

        I_concave=(Vhb>Vhf)+0;

        nf=((1-k)/C*Vhf)**epsilon;
        hf=(1-k)*nf-p*hh;

        nb=((1-k)/C*Vhb)**epsilon;
        hb=(1-k)*nb-p*hh;

        n0=p/(1-k)*hh;
        Vh0=C/(1-k)*n0**(1/epsilon);

        If=(hf>0)+0;
        Ib=(hb<0)+0;

        I0=(1-If-Ib);

        Vh_Upwind=Vhf*If+Vhb*Ib+Vh0*I0;

        n=((1-k)/C*Vh_Upwind)**epsilon;
        u=xx*hh**(1-gamma)/(1-gamma)-C/(1+1/epsilon)*n**(1+1/epsilon);
        
        aaa=np.nan_to_num(hf,nan=0)
        hf=aaa
        aaa=np.nan_to_num(hb,nan=0)
        hb=aaa
    #     where_are_nan = np.isnan(hb)
    #     hb[where_are_nan] = 0
    #     where_are_nan = np.isnan(hf)
    #     hf[where_are_nan] = 0

        X=-np.minimum(hb,0)/dh;
        Y=-np.maximum(hf,0)/dh+np.minimum(hb,0)/dh;
        Z=np.maximum(hf,0)/dh;

        updiag=0;
        for j in range(0,J,1):
            pp=Z[0:I-1,j];
            pp=pp.reshape(I-1,1)
            gg=np.vstack([pp,0]);
            updiag=np.vstack([updiag,gg]);

        Y=np.transpose(Y);
        centdiag=Y.reshape(I*J,1);   #按行来reshape    python中的reshape是按行的，但此处我需要按列来，所以转置

        lowdiag=X[1:I,0];
        lowdiag=lowdiag.reshape(I-1,1);
        for j in range(1,J,1):
            pp=X[1:I,j];
            pp=pp.reshape(I-1,1);
            gg=np.vstack([0,pp]);
            lowdiag=np.vstack([lowdiag,gg]);

        centdiag=np.transpose(centdiag);
        AA_centdiag=spdiags(centdiag,0,I*J,I*J).toarray();

        lowdiag=np.vstack([lowdiag,0]);
        lowdiag=np.transpose(lowdiag);
        AA_lowdiag=spdiags(lowdiag,-1,I*J,I*J).toarray();

        updiag=np.vstack([updiag,0]);
        updiag=np.transpose(updiag);
        AA_updiag=spdiags(updiag,1,I*J,I*J).toarray();

        AA=AA_centdiag+AA_lowdiag+AA_updiag;

        A=AA+Bswitch;

        if np.max(abs(np.sum(A,axis=1)))>10**(-9):   #这个最大值不太一样    
            print("Improper Transition Matrix")
            break;

        B=(1/Delta+rho)*identity(I*J).toarray()-A;
        invB=np.linalg.inv(B);

        u=np.transpose(u);
        u_stacked=copy.deepcopy(u.reshape(I*J,1));
        V=np.transpose(V);
        V_stacked=copy.deepcopy(V.reshape(I*J,1));


        b=u_stacked+V_stacked/Delta;

        invBcopy=copy.deepcopy(invB);
#         bcopy=np.tile(b,(1,I*J))

#         for j in range(0,I*J,1):
#             for i in range(0,I*J,1):
#                 if (np.isnan(bcopy[i,j])==1)|(bcopy[i,j]==np.inf):
#                     if invBcopy[j,i]==0:
#                         bcopy[i,j]=0;
#         V_stacked2=np.zeros((I*J,1))
#         for j in range(0,I*J,1):
#             V_stacked2[j,0]=np.dot(invBcopy[j,:],bcopy[:,j])
#         V_stacked2=np.dot(invBcopy,b)
        bcopy=np.tile(np.transpose(b),(I*J,1))
        index=np.where(np.isnan(bcopy))
        for f in range(0,len(index[0]),1):
            if invBcopy[index[0][f],index[1][f]]==0:
                bcopy[index[0][f],index[1][f]]=0

        index=np.where(np.isinf(bcopy))
        for f in range(0,len(index[0]),1):
            if invBcopy[index[0][f],index[1][f]]==0:
                bcopy[index[0][f],index[1][f]]=0

        V_stacked2=invBcopy*bcopy
        V_stacked2=V_stacked2.sum(axis=1)
        V_stacked2=V_stacked2.reshape(I*J,1)
    
    
        V=copy.deepcopy(V_stacked2.reshape(J,I));
        V=np.transpose(V);  #matlab是按列来进行reshape，python按行来reshape

        Vchange=V-v;

        v=copy.deepcopy(V);

        Vchangecopy=copy.deepcopy(Vchange)
        aaa=np.nan_to_num(Vchangecopy,nan=0,posinf=0,neginf=0)  
        Vchangecopy=aaa
    #     where_are_nan = np.isnan(Vchangecopy)
    #     where_are_inf = np.isinf(Vchangecopy)
    #     Vchangecopy[where_are_nan] = 0
    #     Vchangecopy[where_are_inf] = 0

        distm[m]=np.max(np.max(abs(Vchangecopy),axis=0));
        print(distm[m]);
        print(m);
        if distm[m]<crit:
            print("Value Function Converged, Iteration =");
            print(m);
            print(distm[m]);
            break;
            
    AT=np.transpose(A)
    b5=np.zeros((I*J,1))
    i_fix=5
    b5[i_fix-1,0]=0.1
    row=np.hstack([np.zeros(i_fix-1),1,np.zeros(I*J-i_fix)])
    AT[i_fix-1,:]=row
    invAT=np.linalg.inv(AT);
    gg=np.dot(invAT,b5);

    g_sum=np.dot(np.transpose(gg),np.ones((I*J,1)))*dh*dx

    gg=gg/g_sum
    # g=np.transpose(gg).reshape(I,J)
    ggcopy=copy.deepcopy(gg)
    # ggcopy=np.transpose(ggcopy)
    g=ggcopy.reshape(J,I)
    g=np.transpose(g)
    
    
    T=k*n+p*hh
    
    Tg=T*g
    TFW1=np.dot(np.ones((1,I)),Tg)
    TotalFirmWelfare=np.dot(TFW1,np.ones((J,1)))
    print('次数',i,'k=',k,'p=',p,'FirmWelfare=',TotalFirmWelfare)
    FirmWelfare[i,0]=TotalFirmWelfare

E:\Anaconda1\lib\site-packages\ipykernel_launcher.py:91: RuntimeWarning: divide by zero encountered in power
E:\Anaconda1\lib\site-packages\ipykernel_launcher.py:115: RuntimeWarning: divide by zero encountered in power
E:\Anaconda1\lib\site-packages\ipykernel_launcher.py:122: RuntimeWarning: invalid value encountered in multiply
E:\Anaconda1\lib\site-packages\ipykernel_launcher.py:125: RuntimeWarning: divide by zero encountered in power


0.0
0
Value Function Converged, Iteration =
0
0.0
次数 0 k= 0.9 p= 0.4 FirmWelfare= [[nan]]
0.0
0
Value Function Converged, Iteration =
0
0.0
次数 1 k= 0.9 p= 0.4222222222222222 FirmWelfare= [[nan]]
0.0
0
Value Function Converged, Iteration =
0
0.0
次数 2 k= 0.9 p= 0.4444444444444445 FirmWelfare= [[nan]]
0.0
0
Value Function Converged, Iteration =
0
0.0


KeyboardInterrupt: 

运行时间：3.7 18:38 19:44

## 结果 

In [8]:
print(FirmWelfare)

[[358.76123239]
 [353.05472496]
 [347.96978163]
 [343.16060184]
 [338.85823622]
 [334.64603128]
 [330.94992091]
 [327.5045427 ]
 [324.28409073]
 [321.3150778 ]
 [360.51258346]
 [354.77785699]
 [349.53114906]
 [344.89640133]
 [340.42729839]
 [336.32268557]
 [332.58908973]
 [329.14009157]
 [325.76830108]
 [322.58417446]
 [362.31355943]
 [356.73695136]
 [351.35247173]
 [346.70467027]
 [342.10082956]
 [338.01490859]
 [334.20824614]
 [330.63704469]
 [327.39179794]
 [324.5150697 ]
 [364.59398261]
 [358.67706087]
 [353.53757949]
 [348.50057063]
 [344.03992004]
 [339.92872762]
 [336.11629267]
 [332.84180003]
 [329.44017698]
 [326.0874601 ]
 [366.9027524 ]
 [360.98305451]
 [355.64648788]
 [350.89508783]
 [346.35991118]
 [342.14778789]
 [338.67259414]
 [334.99837048]
 [331.2686445 ]
 [328.82577561]
 [369.57855056]
 [363.87970441]
 [358.49528453]
 [353.49868477]
 [348.8186048 ]
 [344.89308958]
 [341.34882875]
 [337.15370415]
 [334.43183194]
 [330.71073766]
 [372.99197394]
 [367.12810805]
 [361.95

结果存储在这里

## 测试


In [1]:
%%time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys
from scipy.sparse import spdiags
from datetime import datetime
from openpyxl import load_workbook
from scipy.sparse import identity
import copy



k=0.6
p=0.5

gamma=3;
epsilon=-0.2;

rho=0.9;
C=10;

a=1;
eta=0.6;
xmean=a/eta;
sigma=0.8;
sd=sigma**2/2/eta
xmin=xmean-2*sd
xmax=xmean+2*sd

I=1000;
hmin=0;
hmax=100;
h=np.linspace(hmin,hmax,I);  #列向量  
                              #linspace(start,stop,num)生成array对象,num是点的数量，包括两端
h=h.reshape(I,1);
#print(h);
dh=(hmax-hmin)/(I-1);

J=7;
x=np.linspace(xmin,xmax,J);    # 行向量
x=x.reshape(1,J);    #此时x的输出结果为[[2  6  5  4]]，可以用x[0][1]来调取
dx=(xmax-xmin)/(J-1);
dx2=dx**2;

hh=np.dot(h,np.ones((1,J)));
xx=np.dot(np.ones((I,1)),x);

mu=a-x*eta;      
s2=0.5*sigma**2;

Vhf = np.zeros((I,J));
Vhb = np.zeros((I,J));
Vxf = np.zeros((I,J));
Vxb = np.zeros((I,J));
Vxx = np.zeros((I,J));
n = np.zeros((I,J));

chi =  - np.minimum(mu,0)/dx + s2/dx2;              
yy =  np.minimum(mu,0)/dx - np.maximum(mu,0)/dx - s2*2/dx2;
zeta = np.maximum(mu,0)/dx + s2/dx2;

maxit=100;
crit=10**(-6);
Delta=1000;

updiag=np.zeros((I,1));
for j in range(0,J,1):
    gg=np.tile(zeta[0][j],(I,1));
    updiag=np.vstack([updiag,gg]);

centdiag=np.tile(chi[0][0]+yy[0][0],(I,1))
for j in range(1,J-1,1):
    gg=np.tile(yy[0][j],(I,1));
    centdiag=np.vstack([centdiag,gg]);
gg=np.tile(yy[0][J-1]+zeta[0][J-1],(I,1))
centdiag=np.vstack([centdiag,gg])

lowdiag=np.tile(chi[0][1],(I,1));
for j in range(2,J,1):
    gg=np.tile(chi[0][j],(I,1));
    lowdiag=np.vstack([lowdiag,gg]);

centdiag=np.transpose(centdiag);
Bswitch_centdiag=spdiags(centdiag,0,I*J,I*J).toarray();
lowdiag=np.transpose(lowdiag);
Bswitch_lowdiag=spdiags(lowdiag,-I,I*J,I*J).toarray();
updiag=np.transpose(updiag);
Bswitch_updiag=spdiags(updiag,I,I*J,I*J).toarray();
Bswitch=Bswitch_centdiag+Bswitch_lowdiag+Bswitch_updiag;

v0=(xx*hh**(1-gamma)/(1-gamma))/rho;
v=v0;

maxit=30;
distm=np.zeros(maxit);
for m in range(0,maxit,1):   
    V=copy.deepcopy(v);
    Vhf=np.zeros((I,J));
    Vhf[0:I-1,:]=(V[1:I,:]-V[0:I-1,:])/dh;
    Vhf[I-1,:]=(V[I-1,:]-V[I-2,:])/dh;

    Vhb=np.zeros((I,J));
    Vhb[1:I,:]=(V[1:I,:]-V[0:I-1,:])/dh;
    Vhb[0,:]=(V[1,:]-V[0,:])/dh;

    I_concave=(Vhb>Vhf)+0;

    nf=((1-k)/C*Vhf)**epsilon;
    hf=(1-k)*nf-p*hh;

    nb=((1-k)/C*Vhb)**epsilon;
    hb=(1-k)*nb-p*hh;

    n0=p/(1-k)*hh;
    Vh0=C/(1-k)*n0**(1/epsilon);

    If=(hf>0)+0;
    Ib=(hb<0)+0;

    I0=(1-If-Ib);

    Vh_Upwind=Vhf*If+Vhb*Ib+Vh0*I0;

    n=((1-k)/C*Vh_Upwind)**epsilon;
    u=xx*hh**(1-gamma)/(1-gamma)-C/(1+1/epsilon)*n**(1+1/epsilon);

    aaa=np.nan_to_num(hf,nan=0)
    hf=aaa
    aaa=np.nan_to_num(hb,nan=0)
    hb=aaa
#     where_are_nan = np.isnan(hb)
#     hb[where_are_nan] = 0
#     where_are_nan = np.isnan(hf)
#     hf[where_are_nan] = 0

    X=-np.minimum(hb,0)/dh;
    Y=-np.maximum(hf,0)/dh+np.minimum(hb,0)/dh;
    Z=np.maximum(hf,0)/dh;

    updiag=0;
    for j in range(0,J,1):
        pp=Z[0:I-1,j];
        pp=pp.reshape(I-1,1)
        gg=np.vstack([pp,0]);
        updiag=np.vstack([updiag,gg]);

    Y=np.transpose(Y);
    centdiag=Y.reshape(I*J,1);   #按行来reshape    python中的reshape是按行的，但此处我需要按列来，所以转置

    lowdiag=X[1:I,0];
    lowdiag=lowdiag.reshape(I-1,1);
    for j in range(1,J,1):
        pp=X[1:I,j];
        pp=pp.reshape(I-1,1);
        gg=np.vstack([0,pp]);
        lowdiag=np.vstack([lowdiag,gg]);

    centdiag=np.transpose(centdiag);
    AA_centdiag=spdiags(centdiag,0,I*J,I*J).toarray();

    lowdiag=np.vstack([lowdiag,0]);
    lowdiag=np.transpose(lowdiag);
    AA_lowdiag=spdiags(lowdiag,-1,I*J,I*J).toarray();

    updiag=np.vstack([updiag,0]);
    updiag=np.transpose(updiag);
    AA_updiag=spdiags(updiag,1,I*J,I*J).toarray();

    AA=AA_centdiag+AA_lowdiag+AA_updiag;

    A=AA+Bswitch;

    if np.max(abs(np.sum(A,axis=1)))>10**(-9):   #这个最大值不太一样    
        print("Improper Transition Matrix")
        break;

    B=(1/Delta+rho)*identity(I*J).toarray()-A;
    invB=np.linalg.inv(B);

    u=np.transpose(u);
    u_stacked=copy.deepcopy(u.reshape(I*J,1));
    V=np.transpose(V);
    V_stacked=copy.deepcopy(V.reshape(I*J,1));


    b=u_stacked+V_stacked/Delta;

    invBcopy=copy.deepcopy(invB);
#         bcopy=np.tile(b,(1,I*J))

#         for j in range(0,I*J,1):
#             for i in range(0,I*J,1):
#                 if (np.isnan(bcopy[i,j])==1)|(bcopy[i,j]==np.inf):
#                     if invBcopy[j,i]==0:
#                         bcopy[i,j]=0;
#         V_stacked2=np.zeros((I*J,1))
#         for j in range(0,I*J,1):
#             V_stacked2[j,0]=np.dot(invBcopy[j,:],bcopy[:,j])
#         V_stacked2=np.dot(invBcopy,b)
    bcopy=np.tile(np.transpose(b),(I*J,1))
    index=np.where(np.isnan(bcopy))
    for f in range(0,len(index[0]),1):
        if invBcopy[index[0][f],index[1][f]]==0:
            bcopy[index[0][f],index[1][f]]=0

    index=np.where(np.isinf(bcopy))
    for f in range(0,len(index[0]),1):
        if invBcopy[index[0][f],index[1][f]]==0:
            bcopy[index[0][f],index[1][f]]=0

    V_stacked2=invBcopy*bcopy
    V_stacked2=V_stacked2.sum(axis=1)
    V_stacked2=V_stacked2.reshape(I*J,1)


    V=copy.deepcopy(V_stacked2.reshape(J,I));
    V=np.transpose(V);  #matlab是按列来进行reshape，python按行来reshape

    Vchange=V-v;

    v=copy.deepcopy(V);

    Vchangecopy=copy.deepcopy(Vchange)
    aaa=np.nan_to_num(Vchangecopy,nan=0,posinf=0,neginf=0)  
    Vchangecopy=aaa
#     where_are_nan = np.isnan(Vchangecopy)
#     where_are_inf = np.isinf(Vchangecopy)
#     Vchangecopy[where_are_nan] = 0
#     Vchangecopy[where_are_inf] = 0

    distm[m]=np.max(np.max(abs(Vchangecopy),axis=0));
    print(distm[m]);
    print(m);
    if distm[m]<crit:
        print("Value Function Converged, Iteration =");
        print(m);
        print(distm[m]);
        break;

AT=np.transpose(A)
b5=np.zeros((I*J,1))
i_fix=5
b5[i_fix-1,0]=0.1
row=np.hstack([np.zeros(i_fix-1),1,np.zeros(I*J-i_fix)])
AT[i_fix-1,:]=row
invAT=np.linalg.inv(AT);
gg=np.dot(invAT,b5);

g_sum=np.dot(np.transpose(gg),np.ones((I*J,1)))*dh*dx

gg=gg/g_sum
# g=np.transpose(gg).reshape(I,J)
ggcopy=copy.deepcopy(gg)
# ggcopy=np.transpose(ggcopy)
g=ggcopy.reshape(J,I)
g=np.transpose(g)


T=k*n+p*hh

Tg=T*g
TFW1=np.dot(np.ones((1,I)),Tg)
TotalFirmWelfare=np.dot(TFW1,np.ones((J,1)))
print('k=',k,'p=',p,'FirmWelfare=',TotalFirmWelfare)

E:\Anaconda1\lib\site-packages\ipykernel_launcher.py:91: RuntimeWarning: divide by zero encountered in power
E:\Anaconda1\lib\site-packages\ipykernel_launcher.py:115: RuntimeWarning: divide by zero encountered in power
E:\Anaconda1\lib\site-packages\ipykernel_launcher.py:122: RuntimeWarning: invalid value encountered in multiply
E:\Anaconda1\lib\site-packages\ipykernel_launcher.py:125: RuntimeWarning: divide by zero encountered in power


28.25192115715134
0


E:\Anaconda1\lib\site-packages\ipykernel_launcher.py:106: RuntimeWarning: invalid value encountered in greater
E:\Anaconda1\lib\site-packages\ipykernel_launcher.py:117: RuntimeWarning: invalid value encountered in greater
E:\Anaconda1\lib\site-packages\ipykernel_launcher.py:118: RuntimeWarning: invalid value encountered in less


0.8146774443302496
1
0.006617509536531063
2
2.5560696390591886e-06
3
3.525926023373316e-09
4
Value Function Converged, Iteration =
4
3.525926023373316e-09
k= 0.6 p= 0.5 FirmWelfare= [[nan]]
Wall time: 45.7 s


## 改变matlab中translate.m文件中的参数，来减少nan的行数。改变之前的参数为：

gamma=3; %-3会使n和V变得很平   %-2
epsilon=-0.2;%好像不能太大  %-0.05  %epsilon的绝对值稍微大一些，可以使n上升的幅度变大，V也会有一些改变

rho=0.9;%rho变小会使n的变化幅度变小
C=10;%C越小，断点越大，n的幅度也大   %2
k=0.6;%k越小，断点越大   %0.6  3.7晚的值    0.65大
p=0.5;                  %0.5  3.7晚的值  0.55大  0.45小

a=1;
eta=0.6;
xmean=a/eta;
sigma=0.8;
sd=sigma^2/2/eta
xmin=xmean-2*sd
xmax=xmean+2*sd

I=1000;
hmin=0;  %-0.02
hmax=100;
h=linspace(hmin,hmax,I)';   %列向量
dh=(hmax-hmin)/(I-1);

J=7;
x=linspace(xmin,xmax,J);    %行向量
dx=(xmax-xmin)/(J-1);
dx2=dx^2;

## 下面的参数结果还可以接受，但是n的变化幅度不好

gamma=3; %-3会使n和V变得很平   %-2
epsilon=-0.01;%好像不能太大  %-0.05  %epsilon的绝对值稍微大一些，可以使n上升的幅度变大，V也会有一些改变

rho=0.99;%rho变小会使n的变化幅度变小
C=2;%C越小，断点越大，n的幅度也大   %2
k=0.6;%k越小，断点越大   %0.6  3.7晚的值    0.65大
p=0.5;                  %0.5  3.7晚的值  0.55大  0.45小

a=1;
eta=6;
xmean=a/eta;
sigma=0.9;
sd=sigma^2/2/eta
xmin=xmean-2*sd
xmax=xmean+2*sd

I=1000;
hmin=0;  %-0.02
hmax=100;
h=linspace(hmin,hmax,I)';   %列向量
dh=(hmax-hmin)/(I-1);

J=7;
x=linspace(xmin,xmax,J);    %行向量
dx=(xmax-xmin)/(J-1);
dx2=dx^2;